# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download(['punkt', 'wordnet'])

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#imblearn
from imblearn.over_sampling import RandomOverSampler

from sklearn.base import BaseEstimator, TransformerMixin
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from imblearn.under_sampling import RandomUnderSampler

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql_table("messages", con=engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


based on this quick check most of the data is very imbalanced

In [4]:
X = df["message"]
y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [6]:
def tokenize(text):
    '''
    Receives text related data and processes it
    Args: text related data (columns)
    Returns: tokenized text
    '''
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text) 
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
def multi_tester(X, y):
    '''
    Function to create list of fitted models
    Args: training data X and y
    returns: list of the selected fitted models
    '''
    pipe_1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
    ])
    
    pipe_2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(ExtraTreesClassifier(random_state=42)))
    ])
    
    pipe_3 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(GradientBoostingClassifier(random_state=42)))
    ])
    
    pipe_4 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
    ])
    
    pipe_5 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SVC(random_state=42)))
    ])
    
    pips = [pipe_1, pipe_2, pipe_3, pipe_4, pipe_5]
    pip_names = ['RandomForestClassifier', 'ExtraTreesClassifier', 'GradientBoostingClassifier', 
                 'AdaBoostClassifier', 'SVC']
    
    model_fits = []
    for i in range(len(pips)):
        print('Model: ', pip_names[i])
        print(pips[i].get_params())
        mdl = pips[i].fit(X, y)
        model_fits.append(mdl)
        
    return model_fits

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)

In [9]:
fitted_mdls = multi_tester(X_train, y_train)

Model:  RandomForestClassifier
{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x000002C2469CBCA8>,
                vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',

Model:  GradientBoostingClassifier
{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x000002C2469CBCA8>,
                vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                           init=None,
                                                           learning_rate=0.1,
                                                           loss='deviance',
                                                   

Model:  SVC
{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x000002C2469CBCA8>,
                vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                    coef0=0.0, decision_function_shape='ovr',
                                    degree=3, gamma='auto_deprecated',
                                    kernel='rbf', max_iter=-1,
                                    probability=False, random_state=42,
                     

### 5. Test your models
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
target_names = y_train.columns.tolist()

In [11]:
def perf_report(model, X_test, y_test):
    '''
    Function to return model classification reports
    Input: Model list, and test data 
    Output: Prints the Classification report
    '''
    pip_names = ['RandomForestClassifier', 'ExtraTreesClassifier', 'GradientBoostingClassifier', 
             'AdaBoostClassifier', 'SVC']
    
    for i in range(len(model)):
        print('______________________________Model______________________________')
        print('______________________________', pip_names[i], '______________________________')
        y_pred = model[i].predict(X_test)
        print(classification_report(y_test, y_pred, target_names = target_names))

In [12]:
perf_report(fitted_mdls, X_test, y_test)

______________________________Model______________________________
______________________________ RandomForestClassifier ______________________________


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      6534
               request       0.86      0.36      0.50      1472
                 offer       0.00      0.00      0.00        38
           aid_related       0.74      0.53      0.62      3545
          medical_help       0.69      0.07      0.13       701
      medical_products       0.64      0.07      0.12       446
     search_and_rescue       0.59      0.07      0.13       226
              security       0.00      0.00      0.00       160
              military       0.62      0.06      0.11       267
                 water       0.86      0.21      0.34       543
                  food       0.83      0.38      0.52       965
               shelter       0.82      0.21      0.33       775
              clothing       0.82      0.07      0.13       127
                 money       0.75      0.03      0.06       191
        missing_people       1.00      

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      6534
               request       0.00      0.00      0.00      1472
                 offer       0.00      0.00      0.00        38
           aid_related       0.00      0.00      0.00      3545
          medical_help       0.00      0.00      0.00       701
      medical_products       0.00      0.00      0.00       446
     search_and_rescue       0.00      0.00      0.00       226
              security       0.00      0.00      0.00       160
              military       0.00      0.00      0.00       267
                 water       0.00      0.00      0.00       543
                  food       0.00      0.00      0.00       965
               shelter       0.00      0.00      0.00       775
              clothing       0.00      0.00      0.00       127
                 money       0.00      0.00      0.00       191
        missing_people       0.00      

-shops has very little label diversity so it became an edge case, I will drop it for the optimization

______________________________ RandomForestClassifier ______________________________


                           precision    recall  f1-score   support
                           
                           
             micro avg       0.80      0.44      0.57     27308
             
             
             macro avg       0.58      0.16      0.21     27308
             
             
          weighted avg       0.74      0.44      0.50     27308
          
          
           samples avg       0.65      0.42      0.46     27308
           

______________________________ ExtraTreesClassifier ______________________________


             micro avg       0.79      0.44      0.56     27308
             
             
             macro avg       0.53      0.15      0.21     27308
             
             
          weighted avg       0.71      0.44      0.49     27308
          
          
           samples avg       0.66      0.42      0.46     27308

______________________________ GradientBoostingClassifier ______________________________


             micro avg       0.76      0.57      0.65     27308
             
             
             macro avg       0.51      0.32      0.38     27308
             
             
          weighted avg       0.72      0.57      0.61     27308
          
          
           samples avg       0.65      0.50      0.52     27308
           
           
______________________________ AdaBoostClassifier ______________________________


             micro avg       0.77      0.58      0.66     27308
             
             
             macro avg       0.58      0.33      0.40     27308
             
             
          weighted avg       0.73      0.58      0.62     27308
          
          
           samples avg       0.63      0.50      0.51     27308
           
______________________________ SVC ______________________________


             micro avg       0.76      0.24      0.36     27308
             
             
             macro avg       0.02      0.03      0.02     27308
             
             
          weighted avg       0.18      0.24      0.21     27308
          
          
           samples avg       0.76      0.32      0.40     27308


### 6. Improve models based on poor target performance elimination
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.
Testing models after dropping poor predictors

In [13]:
# dropping the targets that had the word performances based on the classification report
targs_drop = ['offer', 'security', 'infrastructure_related', 'tools', 
              'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'fire', 'other_weather']
y_min = y.copy()
y_min.drop(targs_drop, axis = 1, inplace = True)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_min, random_state = 42, test_size = 0.33)

In [15]:
fitted_mdls_min = multi_tester(X_train, y_train)

Model:  RandomForestClassifier
{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x000002C2469CBCA8>,
                vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',

Model:  GradientBoostingClassifier
{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x000002C2469CBCA8>,
                vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                           init=None,
                                                           learning_rate=0.1,
                                                           loss='deviance',
                                                   

Model:  SVC
{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x000002C2469CBCA8>,
                vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                    coef0=0.0, decision_function_shape='ovr',
                                    degree=3, gamma='auto_deprecated',
                                    kernel='rbf', max_iter=-1,
                                    probability=False, random_state=42,
                     

In [16]:
target_names = y_train.columns.tolist()

In [17]:
perf_report(fitted_mdls_min, X_test, y_test)

______________________________Model______________________________
______________________________ RandomForestClassifier ______________________________


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


                   precision    recall  f1-score   support

          related       0.82      0.93      0.87      6534
          request       0.86      0.36      0.50      1472
      aid_related       0.74      0.53      0.62      3545
     medical_help       0.69      0.07      0.13       701
 medical_products       0.64      0.07      0.12       446
search_and_rescue       0.59      0.07      0.13       226
         military       0.62      0.06      0.11       267
            water       0.86      0.21      0.34       543
             food       0.83      0.38      0.52       965
          shelter       0.82      0.21      0.33       775
         clothing       0.82      0.07      0.13       127
            money       0.75      0.03      0.06       191
   missing_people       1.00      0.04      0.07       104
         refugees       0.84      0.09      0.17       293
            death       0.82      0.12      0.20       406
        other_aid       0.44      0.03      0.05      1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



______________________________ RandomForestClassifier ______________________________

                           precision    recall  f1-score   support 
                           
              micro avg       0.80      0.48      0.60     25330
              
              macro avg       0.72      0.22      0.30     25330
              
           weighted avg       0.78      0.48      0.53     25330
           
            samples avg       0.66      0.44      0.48     25330

______________________________ ExtraTreesClassifier ______________________________

        micro avg       0.79      0.46      0.59     25330
        
        macro avg       0.68      0.20      0.27     25330
        
     weighted avg       0.75      0.46      0.52     25330
     
      samples avg       0.65      0.43      0.47     25330    

______________________________ GradientBoostingClassifier ______________________________

        micro avg       0.78      0.61      0.68     25330
        
        macro avg       0.65      0.43      0.50     25330
        
     weighted avg       0.76      0.61      0.65     25330
     
      samples avg       0.66      0.52      0.54     25330
           
           
______________________________ AdaBoostClassifier ______________________________

        micro avg       0.77      0.61      0.69     25330
        
        macro avg       0.69      0.42      0.51     25330
        
     weighted avg       0.75      0.61      0.66     25330
     
      samples avg       0.64      0.51      0.53     25330
           
______________________________ SVC ______________________________

        micro avg       0.76      0.26      0.38     25330
        
        macro avg       0.03      0.04      0.03     25330
        
     weighted avg       0.19      0.26      0.22     25330
     
      samples avg       0.76      0.33      0.41     25330


### 7. Improve your model
Use grid search to find better parameters. 

I will work on my best performing model adaboost and using the reduced target data

In [18]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

In [19]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000002C2469CBCA8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                                      random_state=42),


In [20]:
parameters = {'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__random_state': [42],
             'clf__estimator__learning_rate': [0.5]} 

cv = GridSearchCV(pipeline, param_grid = parameters, cv = 10,
                  refit = True, verbose = 1, return_train_score = True, n_jobs = -1)

In [21]:
cv

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                           

### 8. Test selected model

In [22]:
# dropping the targets that had the word performances based on the classification report
targs_drop = ['offer', 'security', 'infrastructure_related', 'tools', 
              'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'fire', 'other_weather']
y_min = y.copy()
y_min.drop(targs_drop, axis = 1, inplace = True)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y_min, random_state = 42, test_size = 0.33)

In [24]:
best_ada = cv.fit(X_train, y_train)


print('Best model :', best_ada.best_score_)
print('Params :', best_ada.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 28.8min finished


Best model : 0.2847870644500114
Params : {'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 100, 'clf__estimator__random_state': 42, 'tfidf__use_idf': False}


In [25]:
y_pred = best_ada.predict(X_test)
print(classification_report(y_test, y_pred, target_names = target_names))

                   precision    recall  f1-score   support

          related       0.81      0.96      0.88      6534
          request       0.83      0.51      0.63      1472
      aid_related       0.76      0.59      0.67      3545
     medical_help       0.60      0.19      0.28       701
 medical_products       0.75      0.23      0.35       446
search_and_rescue       0.70      0.10      0.18       226
         military       0.64      0.21      0.31       267
            water       0.75      0.60      0.67       543
             food       0.81      0.69      0.75       965
          shelter       0.80      0.50      0.62       775
         clothing       0.70      0.35      0.47       127
            money       0.54      0.19      0.29       191
   missing_people       0.82      0.13      0.23       104
         refugees       0.60      0.20      0.30       293
            death       0.81      0.37      0.51       406
        other_aid       0.62      0.09      0.15      1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 9. Other Approaches


Custom estimators (inspired by: [repo](https://github.com/hnbezz/Portfolio_under_construction/blob/master/Disaster_Response_Pipeline/ML%20Pipeline%20Preparation.ipynb) )

In [26]:
class StartVerbExtractor(BaseEstimator, TransformerMixin):
    def start_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) != 0:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1
        return 0


    def fit(self, X, y=None):
        return self
    

    def transform(self, X):
        X_tag = pd.Series(X).apply(self.start_verb)
        return pd.DataFrame(X_tag)

In [27]:
def get_text_len(data):
    return np.array([len(text) for text in data]).reshape(-1, 1)

In [28]:
# dropping the targets that had the word performances based on the classification report
targs_drop = ['offer', 'security', 'infrastructure_related', 'tools', 
              'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'fire', 'other_weather', 'other_aid']
y_min = y.copy()
y_min.drop(targs_drop, axis = 1, inplace = True)
target_names = y_min.columns.tolist()

In [29]:
#stratifying data
mlss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.33, random_state=42)

for train_index, test_index in mlss.split(X, y_min):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_min.values[train_index], y_min.values[test_index]

In [30]:
y_train = pd.DataFrame(y_train,columns=target_names)
y_test = pd.DataFrame(y_test,columns=target_names)

In [31]:
pipeline_2 = Pipeline([
    ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('best', TruncatedSVD()),
                ('tfidf', TfidfTransformer())])), 
        ('start_verb', StartVerbExtractor())])), 
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

In [32]:
pipeline_2.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,
                                                                    max_fea

In [33]:
parameters = {'clf__estimator__n_estimators': [100, 200, 300], 
              'clf__estimator__random_state': [42],
             'clf__estimator__learning_rate': [0.1]} 

cv_2 = GridSearchCV(pipeline, param_grid = parameters, cv = 10,
                  refit = True, verbose = 1, return_train_score = True, n_jobs = -1)

In [34]:
cv_2

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                           

In [35]:
best_ada_2 = cv_2.fit(X_train, y_train)


print('Best model :', best_ada_2.best_score_)
print('Params :', best_ada_2.best_params_)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 56.2min finished


Best model : 0.2633796401730813
Params : {'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 300, 'clf__estimator__random_state': 42}


In [36]:
y_pred = best_ada_2.predict(X_test)
print(classification_report(y_test, y_pred, target_names = target_names))

                   precision    recall  f1-score   support

          related       0.79      0.98      0.88      6570
          request       0.82      0.49      0.61      1477
      aid_related       0.79      0.54      0.65      3584
     medical_help       0.66      0.14      0.23       688
 medical_products       0.71      0.19      0.30       433
search_and_rescue       0.63      0.11      0.19       239
         military       0.73      0.17      0.28       284
            water       0.79      0.64      0.71       552
             food       0.78      0.72      0.75       965
          shelter       0.86      0.47      0.60       764
         clothing       0.76      0.33      0.46       134
            money       0.74      0.13      0.21       199
   missing_people       0.89      0.16      0.28        98
         refugees       0.71      0.17      0.27       289
            death       0.79      0.38      0.52       394
        transport       0.80      0.16      0.27       

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [37]:
test_text = ['there is a storm and people are trapped']
test = cv_2.predict(test_text)
print(y_train.columns.values[(test.flatten()==1)])

['related' 'weather_related' 'storm']


That is a pretty cool prediction, let's try a few more

In [38]:
test_text = ['we are having an earthquake, buildings are destroyed, victims need clothes']
test = cv_2.predict(test_text)
print(y_train.columns.values[(test.flatten()==1)])

['related' 'request' 'aid_related' 'buildings' 'weather_related'
 'earthquake' 'direct_report']


In [39]:
test_text = ['there was an accident near the bank and we need an ambulance']
test = cv_2.predict(test_text)
print(y_train.columns.values[(test.flatten()==1)])

['related']


### 9. Export your model as a pickle file

In [40]:
pickle.dump(cv_2, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.